<a href="https://colab.research.google.com/github/esemsc-peo23/introduction-to-python/blob/main/Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assessment 1**

For our first assessment, our goal is to solve an imputation problem: we will create a neural network architecture that learns how to recover missing portions of an image.

This is an important problem in magnetic resonance imaging (MRI), where patient scans are often limited to a few areas to avoid lengthy scanning times.

In particular, we are going to focus on images of human heads. We have managed to gain access to one hundred images of patient's heads but, unfortunately, these images have a significant portion of missing information. Your goal during the assessment is to design a neural network that can recover these missing portions.

<br>

---

<br>

We do not have access to the labels for the images we want to recover, so we will have to be a bit creative to obtain a workable dataset on which to train our neural network.

Fortunately for us, we have access to a generative model that has been trained to produce realistic-looking MRI images of patient's heads. Using this model, you will create an appropriate dataset to train your architecture. We have provided you with the basic setup code to start using this generative model in **Question 1** below.

The corrupted images that we want to recover are contained in the numpy file `test_set.npy` of this repository. The file contains 100 patient images with a size of 64x64 pixels.

The architecture that you design in this assessment should use the artificially-generated dataset in order to recover the missing information in the images contained in `test_set.npy`.

<br>

All answers to the assessment should be contained within the structure below, but you are free to add new code and text cells as required to your answers. Read the text for each question and follow the instructions carefully. Answers that do not follow this structure will not be marked. **Do NOT change the name of this file.**

Please, **make sure to execute all your cells and save the result of the execution**. We will only mark cells that have been executed and will not execute any cells ourselves.

<br>

---

<br>

<br><br>

## **Question 1**  (25%)

Using the provided image-generation network, create a dataset of brain images that will later be used to train your chosen architecture.

Given that you will likely want to use this dataset multiple times during training, we recommend that you save the generated images to an appropriate folder in your GDrive.

Once you have generated your dataset, load and display ten of your generated images here.

We have also provided you with some corrupted images in the file `test_set.npy` of this repository. You should also load and display ten of these corrupted images here.

Below, we have provided template code, including some required downloads and installations, so that you can easily use the trained generative model. Sample generation in this model is done using the function `generate`, and is controlled by some input arguments. It is your job to figure out a sensible set of parameters that will produce images that are useful for the requirements of your task.

<br>

In [2]:
#importing necessary packages
import os
import sys
import numpy as np
import matplotlib.pyplot as plt


In [4]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
#extracting the ese_invldm package
!unzip /content/drive/MyDrive/cw1_files.zip -d ./
!chmod +x run.sh
!bash ./run.sh
sys.path.append('/content/ese-invldm')

Archive:  /content/drive/MyDrive/cw1_files.zip
  inflating: ./files/config_training.yml  
  inflating: ./run.sh                
  inflating: ./ese-invldm/setup.py   
  inflating: ./files/default_config.yml  
  inflating: ./ese-invldm/ese_invldm/ese_invldm.py  
  inflating: ./ese-invldm/ese_invldm/__init__.py  
  inflating: ./files/autoencoder/autoencoder_ckpt_latest.pth  
  inflating: ./files/diffusion/diffusion_ckpt_latest.pth  
Cloning the repository from https://github.com/dpelacani/InverseLDM.git...
Cloning into 'InverseLDM'...
remote: Enumerating objects: 1331, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 1331 (delta 167), reused 182 (delta 100), pack-reused 1070 (from 1)
Receiving objects: 100% (1331/1331), 467.16 KiB | 13.35 MiB/s, done.
Resolving deltas: 100% (875/875), done.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━

In [6]:
#creating a directory to save my generated images
save_dir = '/content/drive/MyDrive/generated_images'
os.makedirs(save_dir, exist_ok=True)

In [19]:
# generating the uncorrupted images and saving them to the drive
from ese_invldm import generate

"""
Generates samples using a diffusion-based generative model.

This function leverages a pre-configured diffusion model to produce synthetic samples.
The sampling process supports adjustable parameters for total samples, inference steps, and batch size.
A scheduler and temperature can also be configured to control the sampling behaviour.

Parameters:
    num_samples (int):
        Total number of samples to generate.
    num_inference_steps (int):
        Number of diffusion inference steps.
        The minimum number of steps is 1, but we recommend exploring the range from 10 to 50.
        Please note that more steps will increase quality but also the computational cost. Be careful not
        to burn through your credits by using a very large number of steps!
    batch_size (int):
        Number of samples to process in each batch during sampling.
    scheduler (str, optional):
        Sampling scheduler to use (e.g., "ddim", "ddpm"). You can quickly test which one provides the most appropriate results
        for this task.
    temperature (float, optional):
        Sampling temperature to control randomness, given as a number between 0 and 1. Higher values produce more diverse outputs.
    seed (int, optional):
        Random seed for reproducibility. Defaults to 42.

Returns:
    list:
        A list containing the batches of generated samples, where each sample
        corresponds to a single data instance produced by the diffusion model.

"""

samples = generate(num_samples=5, num_inference_steps=1, batch_size=5, scheduler='ddim', temperature= 0.5)

# Convert samples to a list of numpy arrays for plotting
image_nps = [img.cpu().numpy().squeeze(0) for img in samples]

# Calculate grid dimensions
num_images = 10
rows = 1
cols = 10

# Create figure and subplots
fig, axes = plt.subplots(rows, cols, figsize=(12, 12))

# Flatten axes for easier iteration
axes = axes.flatten()

# Plot images
for i, image in enumerate(image_nps):
    ax = axes[i]
    ax.imshow(image.squeeze(), cmap='gray')  # Display image
    ax.axis('off')  # Hide axes

# Hide any unused subplots
for i in range(num_images, rows * cols):
    axes[i].axis('off')

# Set title
fig.suptitle("Generated Images", fontsize=16)

# Adjust layout and show plot
plt.tight_layout()
plt.show()


#     # Save the image as a PNG file
#     file_path = os.path.join(save_dir, f'image_{i}.png')
#     plt.imsave(file_path, image_np, cmap='gray')

# print(f"Images saved to: {save_dir}")



/content/InverseLDM/invldm/runners/base_runner.py:137: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_states = torch.load(path)


Batch 0


100%|██████████| 1/1 [00:00<00:00,  6.36it/s]


ValueError: cannot select an axis to squeeze out which has size not equal to one

In [18]:
print(samples)

[tensor([[[[0.0300, 0.0660, 0.0778,  ..., 0.0900, 0.0802, 0.0595],
          [0.0388, 0.1171, 0.1974,  ..., 0.2520, 0.1575, 0.0935],
          [0.0883, 0.1366, 0.2365,  ..., 0.3652, 0.1731, 0.0944],
          ...,
          [0.0645, 0.0770, 0.1447,  ..., 0.1499, 0.1063, 0.0917],
          [0.0634, 0.0669, 0.1447,  ..., 0.1146, 0.0612, 0.0591],
          [0.0468, 0.0537, 0.0839,  ..., 0.0665, 0.0463, 0.0383]]],


        [[[0.0317, 0.0337, 0.0315,  ..., 0.0571, 0.0409, 0.0325],
          [0.0323, 0.0378, 0.0409,  ..., 0.0958, 0.0765, 0.0508],
          [0.0323, 0.0488, 0.0934,  ..., 0.1991, 0.1831, 0.1039],
          ...,
          [0.0789, 0.0772, 0.0528,  ..., 0.2642, 0.1648, 0.1057],
          [0.0629, 0.0526, 0.0456,  ..., 0.1616, 0.1423, 0.1110],
          [0.0405, 0.0385, 0.0367,  ..., 0.1289, 0.1338, 0.0964]]],


        [[[0.0280, 0.0318, 0.0326,  ..., 0.0594, 0.0447, 0.0385],
          [0.0377, 0.0499, 0.0440,  ..., 0.0715, 0.0603, 0.0453],
          [0.0659, 0.0869, 0.0677,  .

In [ ]:
for i in range(10):
    plt.imshow(image_np, cmap='gray')  # Displaying the image using the grayscale colormap
    plt.show()

In [ ]:
# Iterate through the first 10 images
for i in range(5):
    # Squeezing the image to remove the extra dimension
    image = samples_1[i]
    plt.imshow(image, cmap='gray')  # Displaying the image using the ygrayscale colormap
    plt.show()


In [20]:
# Loading the test dataset

test_set = np.load('/content/gdrive/MyDrive/test_set.npy')

# Iterate through the first 10 images
for i in range(10):
    # Squeezing the image to remove the extra dimension
    image = np.squeeze(test_set[i])
    plt.imshow(image, cmap='gray')  # Displaying the image using the ygrayscale colormap
    plt.show()



FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/test_set.npy'

<br>

---

<br>

## **Question 2**  (25%)

Using the data generated in **Question 1**, create a PyTorch `TensorDataset` and a `DataLoader` for the training set.

Using the provided corrupted images inside `test_set.npy`, create another `TensorDataset` and a `DataLoader` for the test set.

The training dataset should provide batches of brain images generated in **Question 1** and should corrupt these images appropriately so that they resemble images in the test set. The dataset should also pair each image with its corresponding un-corrupted image as a label.

The test dataset should provide the corrupted images provided, for which no labels are available.

Display here ten images of your training dataset and ten images of your test dataset, and their corresponding labels when available.

<br>



In [1]:
#Importing packages required to corrupt the images
!pip install pycm livelossplot torchinfo -q
%pylab inline

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

from livelossplot import PlotLosses

import matplotlib.animation as animation
import seaborn as sns
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torchinfo import summary
import torchvision
from torchvision import transforms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.6/608.6 kB 15.0 MB/s eta 0:00:00
Populating the interactive namespace from numpy and matplotlib


<br>

---

<br>

## **Question 3** (50%)

Using the dataset created in **Question 2**, design and train an architecture to recover the missing image lines of the provided test dataset.

Once you have trained your architecture, display here ten images of the test set with the recovered lines filled in.

Additionally, save the test data with the missing values filled in into a numpy file called `test_set_nogaps.npy`. These images should be **in the same order** as those in the `test_set.npy` file and should have the same pixel size of 64x64. **Any images not contained in the `test_set_nogaps.npy` file or incorrectly ordered will not be marked.**

You have freedom to choose an architecture that you consider appropriate to solve this problem. However, you will need to train your chosen architecture as part of the assessment: **pre-trained networks are not allowed**.

You will be assessed by the quality of your predictions of the missing data values and additional marks will be given for originality in your network design choices. You should include, as part of your answer, a paragraph explaining the architecture you have chosen and any additional design choices and hyperparameters that have been important to build your solution.

This is an open-book assessment and you are encouraged to use resources online, including  tools like chatGPT. However, make sure to always mention the sources for your code and ideas, including websites, papers, and tools like chatGPT.

<br>


<br>

---

<br>